In [1]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
data_dir = "/Users/pervin0527/upstage-ml-regression-ml11/data/new"
train_data = pd.read_csv(f"{data_dir}/new_train_ver1.csv")
test_data = pd.read_csv(f"{data_dir}/new_test_ver1.csv")

/var/folders/h2/dxdwtry542514hp51hkrx__h0000gn/T/ipykernel_9157/754849947.py:2: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(f"{data_dir}/new_train_ver1.csv")


In [3]:
train_data.isnull().sum()

시군구                             0
번지                              0
본번                              0
부번                              0
아파트명                         2126
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1112839
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         870691
k-전화번호                     870274
k-팩스번호                     872742
단지소개기존clob                1050240
k-세대타입(분양형태)               869563
k-관리방식                     869563
k-복도유형                     869890
k-난방방식                     869563
k-전체동수                     870630
k-전체세대수                    869563
k-건설사(시공사)                 871058
k-시행사                      871254
k-사용검사일-사용승인일              869696
k-연면적                      869563
k-주거전용면적      

In [4]:
test_data.isnull().sum()

시군구                          0
번지                           0
본번                           0
부번                           0
아파트명                        10
전용면적(㎡)                      0
계약년월                         0
계약일                          0
층                            0
건축년도                         0
도로명                          0
해제사유발생일                   9060
등기신청일자                       0
거래유형                         0
중개사소재지                       0
k-단지분류(아파트,주상복합등등)        6582
k-전화번호                    6576
k-팩스번호                    6606
단지소개기존clob                8718
k-세대타입(분양형태)              6562
k-관리방식                    6562
k-복도유형                    6564
k-난방방식                    6562
k-전체동수                    6577
k-전체세대수                   6562
k-건설사(시공사)                6579
k-시행사                     6580
k-사용검사일-사용승인일             6563
k-연면적                     6562
k-주거전용면적                  6562
k-관리비부과면적                 6562
k-전용면적별세대현황(60㎡이하)        6562
k-전용면적별세

In [5]:
train_data['is_test'] = 0
test_data['is_test'] = 1

data = pd.concat([train_data, test_data])
data['is_test'].value_counts() 

0    1118822
1       9272
Name: is_test, dtype: int64

In [6]:
missing_data = data[data[['좌표X', '좌표Y']].isnull().any(axis=1)]
print(len(missing_data))

876232


In [7]:
values = []
for i, (index, row) in enumerate(missing_data.iterrows()):
    city = row['시군구']
    road_name = row['도로명']

    value = ' '.join(city.split(' ')[:-1]) + " " + road_name
    values.append(value)

print(len(values))

values_set = set(values)
print(len(values_set))
print(values_set)

876232
8477
{'서울특별시 서초구 동광로28길 15', '서울특별시 서초구 사평대로 142', '서울특별시 구로구 도림로10길 4', '서울특별시 양천구 목동중앙본로22길 63', '서울특별시 은평구 은평터널로 121-6', '서울특별시 은평구 통일로67길 9', '서울특별시 종로구 통일로14길 35', '서울특별시 강동구 성안로 172', '서울특별시 강남구 학동로73길 33', '서울특별시 서초구 사평대로6길 118', '서울특별시 구로구 디지털로27라길 39-8', '서울특별시 서초구 잠원로 177', '서울특별시 금천구 금하로 720', '서울특별시 동작구 대방동9길 4', '서울특별시 서초구 형촌길 15', '서울특별시 구로구 고척로60길 26', '서울특별시 도봉구 도당로 7', '서울특별시 서초구 강남대로34길 58', '서울특별시 종로구 사직로8길 20', '서울특별시 강동구 구천면로47길 69-18', '서울특별시 강남구 압구정로71길 20', '서울특별시 은평구 응암로16길 8-13', '서울특별시 송파구 위례성대로22나길 9', '서울특별시 구로구 가마산로 284', '서울특별시 은평구 응암로14길 1-15', '서울특별시 은평구 불광로13가길 7-3', '서울특별시 영등포구 대방천로 255', '서울특별시 관악구 대학5길 46', '서울특별시 노원구 섬밭로 196', '서울특별시 관악구 남부순환로172길 41', '서울특별시 관악구 남부순환로248길 78', '서울특별시 양천구 목동중앙북로24길 65', '서울특별시 중랑구 봉우재로20가길 21', '서울특별시 양천구 중앙로55길 13', '서울특별시 서대문구 모래내로 430', '서울특별시 은평구 갈현로36길 15', '서울특별시 구로구 도림로12길 15-3', '서울특별시 송파구 백제고분로45길 27', '서울특별시 성동구 독서당로 242', '서울특별시 동작구 상도로41가길 8', '서울특별시 서초구 방배로34길 89', '서울특별시 양천구 신목로16길 13', '서울특별시 

In [8]:
url = "https://www.findlatlng.org/"
browser = webdriver.Chrome()
browser.get(url)

results = {}
for idx, value in enumerate(values_set):
    print(f"{idx:>08}, 검색어: {value}")

    search_element = browser.find_element(By.CLASS_NAME, "form-control")
    if idx > 0:
        search_element.clear()
    
    search_element.send_keys(value)
    search_element.send_keys(Keys.RETURN)
    time.sleep(1.5)

    address_div = browser.find_element(By.XPATH, '//*[@id="app"]/div/div[3]')
    text_content = address_div.text
    
    lines = text_content.split('\n')
    for line in lines:
        if "위도(Latitude)" in line:
            latitude = line.split(': ')[1].split(' /')[0].strip()
        if "경도(Longitude)" in line:
            longitude = line.split(': ')[2].strip()
    
    # 결과를 딕셔너리에 저장
    results[value] = {'좌표Y': latitude, '좌표X': longitude}
    print(f"좌표X: {longitude}, 좌표Y: {latitude}\n")

00000000, 검색어: 서울특별시 서초구 동광로28길 15
좌표X: 126.995946773488, 좌표Y: 37.4933786716272

00000001, 검색어: 서울특별시 서초구 사평대로 142
좌표X: 127.001060333564, 좌표Y: 37.5003729803604

00000002, 검색어: 서울특별시 구로구 도림로10길 4
좌표X: 126.887685816737, 좌표Y: 37.4860537743215

00000003, 검색어: 서울특별시 양천구 목동중앙본로22길 63
좌표X: 126.87296240258, 좌표Y: 37.5414897658694

00000004, 검색어: 서울특별시 은평구 은평터널로 121-6
좌표X: 126.903644735297, 좌표Y: 37.5902754500152

00000005, 검색어: 서울특별시 은평구 통일로67길 9
좌표X: 126.928456431246, 좌표Y: 37.6107131470046

00000006, 검색어: 서울특별시 종로구 통일로14길 35
좌표X: 126.95792577935, 좌표Y: 37.5761551975538

00000007, 검색어: 서울특별시 강동구 성안로 172
좌표X: 127.136298611505, 좌표Y: 37.5382358420709

00000008, 검색어: 서울특별시 강남구 학동로73길 33
좌표X: 127.046974322468, 좌표Y: 37.5208540179734

00000009, 검색어: 서울특별시 서초구 사평대로6길 118
좌표X: 126.994330362924, 좌표Y: 37.4940431879393

00000010, 검색어: 서울특별시 구로구 디지털로27라길 39-8
좌표X: 126.888777584636, 좌표Y: 37.4866024396611

00000011, 검색어: 서울특별시 서초구 잠원로 177
좌표X: 127.011428731865, 좌표Y: 37.5185975042601

00000012, 검색어: 서울특별시 금천구 금하

In [9]:
# missing_data = missing_data.reset_index(drop=True)
for i, (index, row) in enumerate(missing_data.iterrows()):
    city = row['시군구']
    road_name = row['도로명']
    value = ' '.join(city.split(' ')[:-1]) + " " + road_name

    if value in results:
        y, x = results[value]['좌표Y'], results[value]['좌표X']
        data.loc[index, '좌표X'] = float(x)
        data.loc[index, '좌표Y'] = float(y)

In [10]:
train_df = data[data['is_test'] == 0]
test_df = data[data['is_test'] == 1]

train_df = train_df.drop(columns=['is_test'])
test_df = test_df.drop(columns=['is_test', 'target'])

train_df.to_csv(f'{data_dir}/new_train_ver2.csv', index=False)
test_df.to_csv(f'{data_dir}/new_test_ver2.csv', index=False)

browser.quit()

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:2201: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118822 non-null  object 
 2   본번                      1118822 non-null  float64
 3   부번                      1118822 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [12]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9272 entries, 0 to 9271
Data columns (total 51 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   시군구                     9272 non-null   object 
 1   번지                      9272 non-null   object 
 2   본번                      9272 non-null   float64
 3   부번                      9272 non-null   float64
 4   아파트명                    9262 non-null   object 
 5   전용면적(㎡)                 9272 non-null   float64
 6   계약년월                    9272 non-null   int64  
 7   계약일                     9272 non-null   int64  
 8   층                       9272 non-null   int64  
 9   건축년도                    9272 non-null   int64  
 10  도로명                     9272 non-null   object 
 11  해제사유발생일                 212 non-null    float64
 12  등기신청일자                  9272 non-null   object 
 13  거래유형                    9272 non-null   object 
 14  중개사소재지                  9272 non-null   